In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from skimage import filters
from skimage import feature


In [5]:
# Load images and extract features
images = []
labels = []
train_dir = '/content/drive/MyDrive/Eugenia & Thasniya - Term paper/New Plant Diseases Dataset(Augmented)/train'

for folder in os.listdir(train_dir):
    folder_path = os.path.join(train_dir, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = load_img(file_path, target_size=(256, 256))
        img_array = img_to_array(img)

        # Extract texture features using Gabor filters
        texture_features = filters.gabor(img_array[:, :, 0], frequency=0.1, theta=0)

        # Extract color features using color histograms
        color_features = np.zeros((3, 256))
        for i in range(3):
            color_features[i] = np.histogram(img_array[:, :, i], bins=256)[0]

        # Extract shape features using shape index
        shape_features = feature.shape_index(img_array[:, :, 0])

        # Combine features
        combined_features = np.concatenate((np.real(texture_features[0]).flatten(), color_features.flatten(), shape_features.flatten()))

        images.append(combined_features)
        labels.append(folder)

# Convert lists to numpy arrays
X = np.array(images)
y = np.array(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Train SVM model
svm_model = svm.SVC(kernel='rbf', C=1)
svm_model.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = svm_model.predict(X_test_scaled)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Function to predict the class of a single image
def predict_image(image_path):
    img = load_img(image_path, target_size=(256, 256))
    img_array = img_to_array(img)

    # Extract texture features using Gabor filters
    texture_features = filters.gabor(img_array[:, :, 0], frequency=0.1, theta=0)

    # Extract color features using color histograms
    color_features = np.zeros((3, 256))
    for i in range(3):
        color_features[i] = np.histogram(img_array[:, :, i], bins=256)[0]

    # Extract shape features using shape index
    shape_features = feature.shape_index(img_array[:, :, 0])

    # Combine features
    combined_features = np.concatenate((np.real(texture_features[0]).flatten(), color_features.flatten(), shape_features.flatten()))

    # Impute missing values
    combined_features = imputer.transform([combined_features])

    # Standardize features
    combined_features = scaler.transform(combined_features)
    # Predict the class of the image
    prediction = svm_model.predict(combined_features)


    return prediction

# Test the function
image_path = '/content/drive/MyDrive/Eugenia & Thasniya - Term paper/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_mosaic_virus/Copy of e8900c28-b9e9-41d4-b8f6-5c4be8db1296___PSU_CG 2352_180deg.JPG'
prediction = predict_image(image_path)
print("Predicted class:", prediction)


Classification Report:
                                               precision    recall  f1-score   support

                  Tomato___Septoria_leaf_spot       0.00      0.00      0.00        14
Tomato___Spider_mites Two-spotted_spider_mite       0.19      0.71      0.30         7
                         Tomato___Target_Spot       0.00      0.00      0.00         9
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       0.32      0.70      0.44        10
                 Tomato___Tomato_mosaic_virus       1.00      0.33      0.50        12
                             Tomato___healthy       0.29      0.25      0.27         8

                                     accuracy                           0.30        60
                                    macro avg       0.30      0.33      0.25        60
                                 weighted avg       0.31      0.30      0.24        60

Confusion Matrix:
[[0 7 0 2 0 5]
 [0 5 0 2 0 0]
 [0 5 0 4 0 0]
 [0 3 0 7 0 0]
 [0 5 0 3 4 0]
 [0 1 1 4 0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Predicted class: ['Tomato___Tomato_mosaic_virus']


In [6]:
# Evaluate model
y_pred = svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy: ", accuracy)

Model Accuracy:  0.3
